# Classification with scikit-learn

# Logistic Regression

Similar to the regression problem, except that the values $y$ we now want to predict take on only a small number of discrete values.
Let's focus on the *binary classification* problem in which $y$ can take on only two values, $0$ and $1$.

E.g. if we are trying to build a spam classifier for email, $0$ could mean not spam while $1$, spam.

In theory, we could approach the classification problem ignoring the fact that $y$ is discrete-valued, and use linear regression algorithm to predict $y$ given $X$.
This will perform poorly, and it doesn't make sense to output values larger than $1$ or smaller than $0$.

Logistic Regression uses a different function instead, called *sigmoid function*:

$ g(z) = \frac{\mathrm{1} }{\mathrm{1} + e^{-z}} $


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
matplotlib.rcParams['figure.figsize'] = (12, 8)
matplotlib.rcParams['figure.dpi'] = 100

In [ ]:
x = np.linspace(-10, 10, 100)
y = 1 / ( 1  + np.exp(-x))

plt.plot(x, y)

plt.grid()
plt.yticks(np.arange(0.0, 1.1, 0.1))
plt.title("sigmoid function")
plt.xlabel("z")
plt.ylabel("$g(z)$")

Notice that $g(z)$ tends towards 1 as $z \to \infty$, and $g(z)$ tends towards $0$ as $z \to -\infty$

For classification, we can take a threshold, e.g. $0.5$, everything that is above the threshold, is positive, $y = 1$, while the other values mean $y = 0$.

## Logistic Regression: Titanic dataset

We will use the [Titanic Dataset](https://www.kaggle.com/c/titanic) to illustrate the Logistic Regression.

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
dataset = pd.read_csv('files/data/titanic.csv')
dataset

Each line from train/test dataset is a passenger, and each one has a number of features (12).

All features are described on the [Kaggle challenge page](https://www.kaggle.com/c/titanic/data)

In [ ]:
# Notice the 'object' type of columns
dataset.info()

### Data exploration and processing

Which features are categorical?

Which features are numerical?

Which features contain blank, null or empty values?

In [ ]:
dataset['Sex'] = dataset['Sex'].map({'male':0, 'female':1})

dataset.head()

In [ ]:
X_titanic = dataset[[
    'Pclass',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'Survived' # we will remove this later
]].copy()

X_titanic

In [ ]:
X_titanic.info()

Check for missing values.

In [ ]:
print("We have null values: ", X_titanic.isnull().values.any())
X_titanic.isnull().sum()

In [ ]:
X_titanic['Age']

In [ ]:
X_titanic['Age'].mean()

In [ ]:
X_titanic['Age'].fillna(X_titanic['Age'].mean(), inplace=True)

print("We have null values: ", X_titanic.isnull().values.any())
X_titanic

In [ ]:
X_titanic.isnull().sum()

In [ ]:
sns.pairplot(X_titanic, hue='Survived', palette='tab10')

In [ ]:
sns.countplot(x='Sex', data=X_titanic)

In [ ]:
sns.countplot(x='Survived', data=X_titanic)

In [ ]:
X_titanic[X_titanic['Survived'] == 1.0].describe()

In [ ]:
y_titanic = X_titanic['Survived'].copy()
X_titanic.drop(['Survived'], axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_titanic, y_titanic, test_size=0.3)

In [ ]:
X_titanic.shape

In [ ]:
X_train.shape

In [ ]:
X_train[:10]

In [ ]:
type(X_train)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_valid, y_valid)

In [ ]:
model.predict_proba(X_valid)[:10]

# Naive Bayes

Intuition: Naive Bayes solves the classification problem by relying on Bayes's theorem, which is an equation describing the relationship of conditional probabilities of statistical quantities.
In other words, what is the probability of a label, given some features, $P(L~|~{\rm features})$

It's naive because it makes naive assumptions about the distribution of the data.

$ P(L~|~{\rm features}) = \frac{P({\rm features}~|~L)P(L)}{P({\rm features})} $
<br/>
<br/>

If we have only two possible labels (binary classification):
$ \frac{P(L_1~|~{\rm features})}{P(L_2~|~{\rm features})} = \frac{P({\rm features}~|~L_1)}{P({\rm features}~|~L_2)}\frac{P(L_1)}{P(L_2)} $
<br/>
<br/>

But how to compute $ P({\rm features}~|~L_i) $ ?

## Gaussian Naive Bayes

Perhaps the easiest naive Bayes classifier to understand is Gaussian naive Bayes.
In this classifier, the assumption is that data from each label is drawn from a simple Gaussian distribution.
Imagine that you have the following data:

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(100, 2, centers=2, random_state=2, cluster_std=1.5)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=70, cmap='RdBu')

One extremely fast way to create a simple model is to assume that the data is described by a Gaussian distribution with no covariance between dimensions.
This model can be fit by simply finding the mean and standard deviation of the points within each label, which is all you need to define such a distribution.
The result of this naive Gaussian assumption is shown in the following figure:

![image](https://drive.google.com/uc?export=view&id=14P1QVJIQohKvYBHUnKxHW7utSPVgW3N1)


The ellipses here represent the Gaussian generative model for each label, with larger probability toward the center of the ellipses. 

This procedure is implemented in Scikit-Learn's `sklearn.naive_bayes.GaussianNB` estimator:

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X, y)

Generate some data and predict the label

In [ ]:
rng = np.random.RandomState(0)

Xnew = [-6, -14] + [14, 18] * rng.random_sample((5000, 2))

ynew = model.predict(Xnew)

Now we can plot this new data to get an idea of where the decision boundary is

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=70, cmap='RdBu',alpha=1)
lim = plt.axis()

plt.scatter(Xnew[:, 0], Xnew[:, 1], c=ynew, s=20, cmap='RdBu', alpha=0.1)
plt.axis(lim)

### When to use Naive Bayes

Because naive Bayesian classifiers make such stringent assumptions about data, they will generally not perform as well as a more complicated model.
That said, they have several advantages:
* They are extremely fast for both training and prediction
* They provide straightforward probabilistic prediction
* They are often very easily interpretable
* They have very few (if any) tunable parameters
* These advantages mean a naive Bayesian classifier is often a good choice as an initial baseline classification.
If it performs suitably, then congratulations: you have a very fast, very interpretable classifier for your problem. If it does not perform well, then you can begin exploring more sophisticated models, with some baseline knowledge of how well they should perform.

Naive Bayes classifiers tend to perform especially well in one of the following situations:
* When the naive assumptions actually match the data (very rare in practice)
* For very well-separated categories, when model complexity is less important
* For very high-dimensional data, when model complexity is less important

The last two points seem distinct, but they actually are related: as the dimension of a dataset grows, it is much less likely for any two points to be found close together (after all, they must be close in every single dimension to be close overall).
This means that clusters in high dimensions tend to be more separated, on average, than clusters in low dimensions, assuming the new dimensions actually add information.
For this reason, simplistic classifiers like naive Bayes tend to work as well or better than more complicated classifiers as the dimensionality grows: once you have enough data, even a simple model can be very powerful.

# k Nearest Neighbors

k Nearest Neighbors is one of the simplest learning strategies.
Given a new, unknown observation, look up in your reference database which ones have the closest features and assign the predominant class.

A big disadvantage is that you need to store the entire *training* set (it's not actually training since you just keep the values around).

In [ ]:
from sklearn import neighbors, datasets

iris = datasets.load_iris() # just another way to load the iris dataset

In [ ]:
print(iris.DESCR)

In [ ]:
X_iris, y_iris = iris.data, iris.target

X_train, X_valid, y_train, y_valid = train_test_split(X_iris, y_iris, test_size=0.3)

In [ ]:
X_iris[:10]

In [ ]:
y_iris[:100]

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

In [ ]:
knn.score(X_train, y_train)

In [ ]:
knn.score(X_valid, y_valid)

In [ ]:
# What kind of iris has 3cm x 5cm sepal and 4cm x 2cm petal?
result = knn.predict([[3, 5, 4, 2]])
print(result)
print(iris.target_names[result])

Probabilistic predictions

In [ ]:
print(iris.target_names)

In [ ]:
from matplotlib.colors import ListedColormap

# Create color maps for 3-class classification problem, as with iris
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

def plot_iris_knn(x):
    iris = datasets.load_iris()
    X = iris.data[:, :2]  # we only take the first two features. We could
                          # avoid this ugly slicing by using a two-dim dataset
    y = iris.target

    knn = neighbors.KNeighborsClassifier(n_neighbors=x)
    knn.fit(X, y)

    x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
    y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    
    plt.figure()
    plt.grid(False)

    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
    plt.xlabel('sepal length (cm)')
    plt.ylabel('sepal width (cm)')
    plt.axis('tight')

In [ ]:
plot_iris_knn(x=10)

# Decision Trees

Decision trees are extremely intuitive ways to classify or label objects: you simply ask a series of questions designed to zero-in on the classification.
For example, if you wanted to build a decision tree to classify an animal, you could do the following:

![image](https://drive.google.com/uc?export=view&id=13D_mcYQ1cVlB0xfjK2TCYk6eFqwS7pKq)

The binary splitting makes this extremely efficient: in a well-constructed tree, each question will cut the number of options by approximately half, very quickly narrowing the options even among a large number of classes.

Decision Trees end up overfitting the data but we can do better if we combine multiple Decision Trees and average their results.

## Random Forests

A Random Forest is an ensemble of Decision Trees.

In [ ]:
# using the titanic dataset again
X_train, X_valid, y_train, y_valid = train_test_split(X_titanic, y_titanic, test_size=0.3) 

X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()

rf_model.fit(X_train, y_train)

In [ ]:
rf_model.score(X_train, y_train)

In [ ]:
rf_model.score(X_valid, y_valid)

# References

* https://github.com/jakevdp/PythonDataScienceHandbook
* https://github.com/ipython-books/cookbook-2nd-code/blob/master/chapter08_ml/02_titanic.ipynb
* https://github.com/ipython-books/cookbook-2nd/blob/master/chapter08_ml/02_titanic.md
* https://www.kaggle.com/startupsci/titanic-data-science-solutions
* https://datascienceplus.com/logistic-regression-with-python-using-titanic-data/
* https://www.kaggle.com/mnassrib/titanic-logistic-regression-with-python
* https://mashimo.wordpress.com/2018/03/31/logistic-regression-using-sklearn/